In [ ]:
################################################################################## Documents Loader #############################################################################################################################

from langchain_community.document_loaders import (
    PyPDFLoader, TextLoader, Docx2txtLoader, UnstructuredExcelLoader,
    CSVLoader, JSONLoader, UnstructuredPowerPointLoader,
    UnstructuredHTMLLoader, UnstructuredMarkdownLoader, UnstructuredRTFLoader,
    UnstructuredXMLLoader, UnstructuredEmailLoader, UnstructuredEPubLoader,
    UnstructuredImageLoader,
)
from pathlib import Path
import logging
logging.getLogger("pypdf").setLevel(logging.ERROR)

def loadDocs(dir: str):

    data_path = (Path.cwd() / dir).resolve()
    print(f"[DEBUG] Data path: {data_path}")
    documents = []

    if not data_path.exists():
        print("[ERROR] Folder not found!")
        return []

    # ✅ One mapping — extension(s) → loader class
    FILE_LOADERS = {
        PyPDFLoader:                  ["*.pdf"],
        Docx2txtLoader:               ["*.docx", "*.doc"],
        UnstructuredExcelLoader:      ["*.xlsx", "*.xls"],
        CSVLoader:                    ["*.csv"],
        UnstructuredPowerPointLoader: ["*.pptx", "*.ppt"],
        UnstructuredHTMLLoader:       ["*.html", "*.htm"],
        UnstructuredMarkdownLoader:   ["*.md", "*.markdown"],
        UnstructuredRTFLoader:        ["*.rtf"],
        UnstructuredXMLLoader:        ["*.xml"],
        UnstructuredEmailLoader:      ["*.eml", "*.msg"],
        UnstructuredEPubLoader:       ["*.epub"],
        UnstructuredImageLoader:      ["*.png", "*.jpg", "*.jpeg", "*.tiff", "*.bmp"],
        TextLoader:                   ["*.txt", "*.yaml", "*.yml"],
    }
    

    # ✅ Single loop handles everything
    for loader_class, extensions in FILE_LOADERS.items():
        files = []
        for ext in extensions:
            #[1.pdf , 2.pdf]
            files.extend(data_path.glob(f'**/{ext}'))
            
        print(f"[DEBUG] Found {len(files)} {extensions} files")

        for file in files:
            try:
                # pypdf(1.pdf) then pypdf(1.pdf) and so on
                loader = loader_class(str(file))
                documents.extend(loader.load())
            except Exception as e:
                print(f"[ERROR] {file.name}: {e}")

    print(f"\n Total documents loaded: {len(documents)}")
    return documents


# Useage
docs = loadDocs(r"..\data")

if len(docs) > 0:
    print(docs[0].page_content[:100])
    print(docs[0].metadata)
else:
    print("No documents loaded — check errors above!")

In [ ]:
################################################################################## CHUNKING #############################################################################################################################

from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import hashlib

CHUNK_CONFIG = {
    "pdf": (500, 100), "docx": (500, 100), "txt": (400, 80),
    "csv": (300, 50),  "xlsx": (300, 50),  "json": (300, 50),
    "pptx": (400, 80), "html": (500, 100), "md": (500, 100),
    "py": (1500, 200), "js": (1500, 200),
}

def cleanText(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)            # extra spaces/tabs
    text = re.sub(r'\n{3,}', '\n\n', text)      # 3+ newlines → 2
    return text.strip()

def chunkDocs(docs):
    if not docs:
        return []

    chunks = []
    for doc in docs:
        # clean
        doc.page_content = cleanText(doc.page_content)

        # get config based on file type
        ext = doc.metadata.get("source", "").split(".")[-1].lower()
        size, overlap = CHUNK_CONFIG.get(ext, (500, 100))

        # split
        splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)
        doc_chunks = splitter.split_documents([doc])

        # filter garbage + add metadata
        for chunk in doc_chunks:
            if len(chunk.page_content.strip()) > 50:
                chunk.metadata["chunk_id"] = len(chunks)
                chunk.metadata["file_type"] = ext
                chunk.metadata["hash_id"] = hashlib.md5(chunk.page_content.encode()).hexdigest()
                chunks.append(chunk)

    print(f"✅ {len(chunks)} chunks from {len(docs)} docs")
    return chunks

chunks = chunkDocs(docs)
if len(chunks) > 0:
    print(chunks[0].page_content[:100])
    print(chunks[0].metadata)

In [ ]:
from sentence_transformers import SentenceTransformer
from typing import List
from langchain.schema import Document

model = SentenceTransformer("BAAI/bge-large-en-v1.5")
seen_hashes = set()

def EmbeddingChunkedData(chunks: List[Document]):
    
    if not chunks:
        print("[WARNING] No chunks to embed!")
        return [], []

    # filter duplicates using hash from metadata
    unique_chunks = []
    skipped = 0
    for chunk in chunks:
        hash_id = chunk.metadata.get("hash_id")
        if hash_id in seen_hashes:
            skipped += 1
            continue
        seen_hashes.add(hash_id)
        unique_chunks.append(chunk)

    print(f"✅ Unique : {len(unique_chunks)} | 🔁 Skipped : {skipped}")

    if not unique_chunks:
        print("[WARNING] All chunks were duplicates!")
        return [], []

    # embed
    embeddings = model.encode(
        [chunk.page_content for chunk in unique_chunks],
        batch_size=64,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    print(f"✅ Embedded {len(embeddings)} chunks")
    print(f"📐 Shape: {len(embeddings)} x {len(embeddings[0])}")
    
    return unique_chunks, embeddings


# USEAGE
unique_chunks, embeddings = EmbeddingChunkedData(chunks)

# verify
print("\nSample chunk:")
print(unique_chunks[0].page_content[:300])
print("\nMetadata:")
print(unique_chunks[0].metadata)

In [ ]:
import faiss
import numpy as np
import pickle
import os
from typing import List
from langchain.schema import Document

STORE_DIR   = "vector_store"
INDEX_PATH  = f"{STORE_DIR}/index.faiss"
CHUNKS_PATH = f"{STORE_DIR}/chunks.pkl"

# SAVE
def saveVectorStore(chunks: List[Document], embeddings: np.ndarray):
    os.makedirs(STORE_DIR, exist_ok=True)
    embeddings = np.array(embeddings).astype("float32")

    # load existing or create new
    if os.path.exists(INDEX_PATH):
        index = faiss.read_index(INDEX_PATH)
        with open(CHUNKS_PATH, "rb") as f:
            existing_chunks = pickle.load(f)
        existing_hashes = {c.metadata.get("hash_id") for c in existing_chunks}
    else:
        index = faiss.IndexFlatIP(embeddings.shape[1])
        existing_chunks = []
        existing_hashes = set()

    # filter duplicates
    filtered = [(c, e) for c, e in zip(chunks, embeddings)
                if c.metadata.get("hash_id") not in existing_hashes]

    if not filtered:
        print("✅ No new chunks — everything already stored!")
        return

    new_chunks, new_embeddings = zip(*filtered)

    index.add(np.array(new_embeddings).astype("float32"))
    all_chunks = existing_chunks + list(new_chunks)

    faiss.write_index(index, INDEX_PATH)
    with open(CHUNKS_PATH, "wb") as f:
        pickle.dump(all_chunks, f)

    print(f"✅ Added: {len(new_chunks)} | Total: {index.ntotal} vectors")

# Usage
saveVectorStore(unique_chunks, embeddings)


In [ ]:
# LOAD
def loadVectorStore():
    if not os.path.exists(INDEX_PATH):
        print("[ERROR] No vector store found!")
        return None, []
    index = faiss.read_index(INDEX_PATH)
    with open(CHUNKS_PATH, "rb") as f:
        chunks = pickle.load(f)
    print(f"✅ Loaded {index.ntotal} vectors")
    return index, chunks


# SEARCH
def search(query: str, top_k: int = 3):
    index, chunks = loadVectorStore()
    if index is None:
        return []

    query_emb = model.encode([query], normalize_embeddings=True).astype("float32")
    scores, indices = index.search(query_emb, top_k)

    return [
    {
        "content": chunks[i].page_content,       # actual text
        "score":   round(float(s), 4),            # similarity score
        "source":  chunks[i].metadata.get("source") # which file
    }
    for s, i in zip(scores[0], indices[0])  # pair each score with its index
    if i != -1                               # -1 means no result found → skip
    ]
    
results = search("can you tell me how much root gap is allowed as per the saes?")
for i, r in enumerate(results):
    print(f"\n--- Result {i+1} | Score: {r['score']} ---")
    print(r["content"][:300])
    print("Source:", r["source"])


In [ ]:

import requests
from typing import List

from dotenv import load_dotenv
import os

load_dotenv() 

API_KEY = os.getenv("OPENROUTER_API_KEY")
API_URL = os.getenv("API_URL")

SYSTEM_PROMPT = """You are a technical engineering assistant.
Answer ONLY from the context provided.
If answer is not in context, say "Not found in the documents."
Do NOT use your own knowledge."""

def GenerateAnswer(query: str, results: List[dict]) -> str:

    # ✅ lower threshold — 0.35 is fine for technical docs
    top_results = [r for r in results if r["score"] > 0.35]

    if not top_results:
        return "No relevant documents found."

    # ✅ clean context format — just text + source, no dict noise
    context = "\n\n".join([
        f"Source: {r['source'].split(chr(92))[-1]}\n{r['content']}"
        for r in top_results
    ])

    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer strictly from the context:"

    payload = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": prompt}
        ],
        "max_tokens": 500,
        "temperature": 0.1
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "http://localhost",
        "X-Title": "RAG App"
    }

    try:
        response = requests.post(API_URL, json=payload, headers=headers)
        result = response.json()

        if "error" in result:
            print(f"[ERROR] API: {result['error']}")
            return None

        return result["choices"][0]["message"]["content"]

    except Exception as e:
        print(f"[ERROR] Request failed: {e}")
        return None
    
# USEAGE
question = "can you tell me about seas-108?"
results = search(question)
answer  = GenerateAnswer(question, results)
print(answer)

[DEBUG] API_KEY: set
[DEBUG] API_URL: https://openrouter.ai/api/v1/chat/completions
